# Othello Probcut

### Initiale Konfiguration

Importieren von Abhängigkeiten und Konfiguration

In [14]:
import othello_gui
import math
import copy

from tkinter import TclError
from othello import *

# STRATEGIES
HUMAN = 'H'
TRIVIAL = 'T'
MINIMAX = 'M'
ALPHABETA = 'A'
PROBCUT = 'P'

# PLAYERS
BLACK_PLAYER = HUMAN
WHITE_PLAYER = ALPHABETA

### Hilfsfunktionen
Diese Hilfsfunktionen werden von mehreren KI's verwendet

In [8]:
def append_move_if_possible(state, row, col, possible_moves):
    if state.current_board[row][col] == NONE:
        for direction in state._adjacent_opposite_color_directions(row, col, state.turn):
            if state._is_valid_directional_move(row, col, direction[0], direction[1], state.turn):
                possible_moves.append((row, col))
                break
                
def get_possible_moves(state):
    possible_moves = []
    for row in range(state.rows):
        for col in range(state.cols):
            append_move_if_possible(state, row, col, possible_moves)
    return possible_moves

def get_possible_moves_ordered(state):
    possible_moves = []
    
    r_max = state.rows - 1
    c_max = state.cols - 1
    
    # corners
    append_move_if_possible(state, 0    , 0    , possible_moves)
    append_move_if_possible(state, 0    , c_max, possible_moves)
    append_move_if_possible(state, r_max, 0    , possible_moves)
    append_move_if_possible(state, r_max, c_max, possible_moves)
    
    # edges
    for row in range(1, state.rows - 1):
        append_move_if_possible(state, row, 0    , possible_moves)
        append_move_if_possible(state, row, c_max, possible_moves)
        
    for col in range(1, state.cols - 1):
        append_move_if_possible(state, 0    , col, possible_moves)
        append_move_if_possible(state, r_max, col, possible_moves)
    
    # rest of cells
    for row in range(1, state.rows - 1):
        for col in range(1, state.cols - 1):
            append_move_if_possible(state, row, col, possible_moves)
    return possible_moves

def terminal_utility(state):
    winner = state.return_winner()
    if(winner is None):
        return 0
    else:
        return 1 if winner == state.turn else -1
    
def heuristic_utility(state):
    player_cells = state.get_total_cells(state.turn)
    opponent_cells = state.get_total_cells(state._opposite_turn(state.turn))
    return (player_cells - opponent_cells) / (state.rows * state.cols)

### Triviale KI
Diese KI iteriert reihenweise über das Spielfeld und spielt den ersten gültigen Zug

In [3]:
def trivial_ai_make_move(state):
    for row in range(state.rows):
        for col in range(state.cols):
            try:
                state.move(row, col)
                return
            except InvalidMoveException:
                pass


### Minimax KI
Diese KI verwended den Minimax Algorithmus

In [4]:
MINIMAX_DEPTH_LIMIT = 5

def minimax_max(state, depth):
    global minimax_best_move
    if(state.is_game_over()):
        return terminal_utility(state)
    if(depth >= MINIMAX_DEPTH_LIMIT):
        return heuristic_utility(state)
    max_utility = -math.inf
    for move in get_possible_moves(state):
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = minimax_min(tmp_state, depth + 1)
        if(tmp_utility > max_utility):
            max_utility = tmp_utility
            if(depth == 0):
                minimax_best_move = move
    return max_utility

def minimax_min(state, depth):
    global minimax_best_move
    if(state.is_game_over()):
        return -terminal_utility(state)
    if(depth >= MINIMAX_DEPTH_LIMIT):
        return -heuristic_utility(state)
    min_utility = math.inf
    for move in get_possible_moves(state):
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = minimax_max(tmp_state, depth + 1)
        if(tmp_utility < min_utility):
            min_utility = tmp_utility
            if(depth == 0):
                minimax_best_move = move
    return min_utility


def minimax_ai_make_move(state):
    minimax_max(state, 0)
    state.move(minimax_best_move[0], minimax_best_move[1])

### Alpha-Beta KI
Diese KI verwended den Minimax Algorithmus mit Alpha-Beta Pruning

In [5]:
ALPHABETA_DEPTH_LIMIT = 7

def alphabeta_max(state, alpha, beta, depth):
    global alphabeta_best_move
    if(state.is_game_over()):
        return terminal_utility(state)
    if(depth >= ALPHABETA_DEPTH_LIMIT):
        return heuristic_utility(state)
    max_utility = -math.inf
    for move in get_possible_moves_ordered(state):
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = alphabeta_min(tmp_state, alpha, beta, depth + 1)
        if(tmp_utility > max_utility):
            max_utility = tmp_utility
            if(depth == 0):
                alphabeta_best_move = move
        if(max_utility >= beta):
            return max_utility
        alpha = max(alpha, max_utility)
    return max_utility

def alphabeta_min(state, alpha, beta, depth):
    global alphabeta_best_move
    if(state.is_game_over()):
        return -terminal_utility(state)
    if(depth >= ALPHABETA_DEPTH_LIMIT):
        return -heuristic_utility(state)
    min_utility = math.inf
    for move in get_possible_moves_ordered(state):
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = alphabeta_max(tmp_state, alpha, beta, depth + 1)
        if(tmp_utility < min_utility):
            min_utility = tmp_utility
            if(depth == 0):
                alphabeta_best_move = move
        if(min_utility <= alpha):
            return min_utility
        beta = min(beta, min_utility)
    return min_utility

def alphabeta_ai_make_move(state):
    alphabeta_max(state, -math.inf, math.inf, 0)
    state.move(alphabeta_best_move[0], alphabeta_best_move[1])

### ProbCut KI
An dieser Stelle beginnt die Implementierung der Künstlichen Intelligenz mittels des Minimax Algorithmus und ProbCut

In [6]:
def probcut_ai_make_move(state):
    for row in range(state.rows):
        for col in range(state.cols):
            try:
                state.move(row, col)
                return
            except InvalidMoveException:
                pass

### Applikation Starten

Führen sie zum Started der Applikations den folgenden Code aus.

In [16]:
print('Application start') 
gui = othello_gui.OthelloGUI()

try:
    while 1:
        gui._root_window.update()
        game_state = gui._game_state
        if (not game_state.is_game_over()):
            # Check if/which AI is playing
            strat = BLACK_PLAYER if game_state.turn == BLACK else WHITE_PLAYER
            if (strat != HUMAN): 
                # Execute AI strategy for current player
                if(strat == TRIVIAL):
                    trivial_ai_make_move(game_state)
                elif(strat == MINIMAX):
                    minimax_ai_make_move(game_state)
                elif(strat == ALPHABETA):
                    alphabeta_ai_make_move(game_state)
                elif(strat == PROBCUT):
                    probcut_ai_make_move(game_state)
                else:
                    raise Exception('Invalid Strategy')
                    
                # Update GUI after AI move
                gui._board.update_game_state(game_state)
                gui._board.redraw_board()
                gui._black_score.update_score(game_state)
                gui._white_score.update_score(game_state)

                if game_state.is_game_over():
                    gui._player_turn.display_winner(game_state.return_winner())
                else:
                    gui._player_turn.switch_turn(game_state)
except TclError:
    print('Application exit')
except Exception as e:
    gui._root_window.destroy()
    raise e

Application start
Application exit


### Testing code

In [25]:
test_board = OthelloGame(8, 8, BLACK, WHITE, MOST_CELLS)

In [44]:
minimax_ai_make_move(test_board)
test_board.current_board

[['.', '.', '.', 'B', '.', '.', '.', '.'],
 ['.', '.', '.', 'B', 'W', '.', '.', '.'],
 ['.', '.', '.', 'W', '.', '.', '.', '.'],
 ['.', '.', 'W', 'B', 'W', '.', '.', '.'],
 ['W', 'W', 'W', 'B', 'W', 'W', '.', '.'],
 ['.', '.', '.', '.', 'B', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.']]

In [ ]:
import cProfile

cProfile.run('minimax_ai_make_move(test_board)')
test_board.current_board